# Derivatives of the manipulator
In a first time, we come back to the manipulator robot, with a nice Euclidean configuration space.

In [1]:
from robots import loadTalosArm
from scipy.optimize import fmin_slsqp
import pinocchio
from pinocchio.utils import *
from numpy.linalg import norm,inv,pinv,eig,svd

m2a = lambda m: np.array(m.flat)
a2m = lambda a: np.matrix(a).T

robot   = loadTalosArm()
robot.initDisplay(loadModel=True)
robot.viewer.gui.deleteNode('world',True)
robot.initDisplay(loadModel=True)

The objective is to set up the derivatives of the problem defined in arm3dconstraint.py.

## Derivative of the cost
Here nothing special to do: we have taken the sum of square, then the gradient of the cost is simply the residuals. 

## Checking with finite differencing
A rule of thumb is to always first implement the finite-diff of your problem, because it should gives you a good idea of wether the problem is nicely setup and has a chance to work, but also because you will need your finite diff to check the derivatives. 

In [19]:
# %load dcost.py
refQ = robot.q0

def cost(q):
    residuals = m2a(q-refQ)
    return .5*sum(residuals**2)

def dCost(q):
    dq = m2a(q-refQ)
    return dq
  
def numdiffCost(q,h=1e-6):
    f0 = cost(q)
    nq,nf = len(q),1
    dq = zero(nq)
    df_dq = zero([nf,nq])
    for i in range(nq):
        dq[i] = h
        df_dq[:,i] = (cost(q+dq)-f0)/h
        dq[i] = 0
    return df_dq

q=rand(robot.model.nq)
norm(dCost(q)-numdiffCost(q))

1.322917781089311e-06

## Derivative of the log residual
The residual is a composition of two functions: log and M. 
$residual(q) = log(M(q))$

The derivative of the first function is implemented in pinocchio as pinocchio.Jlog.

The derivative of the second function is the Jacobian of the corresponding frame, computed locally (i.e. the velocity nu resulting from the Jacobian are expressed in the local frame at the center of the local frame). To get the frame jacobian, it is necessary to first precompute the joint jacobians, then update the frame placement, before getting the correct frame jacobian.


In [21]:
LOCAL = pinocchio.ReferenceFrame.LOCAL
WORLD = pinocchio.ReferenceFrame.WORLD

pinocchio.forwardKinematics(robot.model,robot.data,q)
pinocchio.computeJointJacobians(robot.model,robot.data,q)
pinocchio.updateFramePlacements(robot.model,robot.data)
pinocchio.getFrameJacobian(robot.model,robot.data,26,LOCAL)        

matrix([[ 3.12464410e-01, -1.73377393e-01, -4.04185585e-01,
         -1.67398617e-01, -9.26937848e-05, -2.13985000e-01,
          0.00000000e+00],
        [ 5.47850180e-01, -1.01596098e-01,  8.15639506e-02,
         -2.61691665e-02,  2.13926647e-01,  0.00000000e+00,
          0.00000000e+00],
        [-2.19364890e-01,  3.75475328e-02,  2.89745219e-02,
          1.87791881e-02,  5.11399160e-02,  3.20000000e-02,
          0.00000000e+00],
        [ 8.46868449e-01, -1.80438502e-01,  1.19973377e-01,
         -1.46452994e-03,  9.99998357e-01,  1.66533454e-16,
          0.00000000e+00],
        [-3.09985293e-01,  5.96857724e-01,  8.02261422e-01,
          5.89190132e-01,  1.11022302e-16,  1.00000000e+00,
          0.00000000e+00],
        [ 4.32114509e-01,  7.81794605e-01, -5.84793125e-01,
          8.07993097e-01,  1.81254957e-03,  1.11022302e-16,
          0.00000000e+00]])

## Derivative in an optimization program
Here is the final optimization program with derivatives of the cost and contraint.

In [ ]:
# %load arm3dconstraint_diff.py
from robots import loadTalosArm
from scipy.optimize import fmin_slsqp
import pinocchio
from pinocchio.utils import *
from numpy.linalg import norm,inv,pinv,eig,svd

m2a = lambda m: np.array(m.flat)
a2m = lambda a: np.matrix(a).T
LOCAL = pinocchio.ReferenceFrame.LOCAL
WORLD = pinocchio.ReferenceFrame.WORLD

robot   = loadTalosArm()
robot.initDisplay(loadModel=True)

class OptimProblem:
    def __init__(self,rmodel,gview=None):
        self.rmodel = rmodel
        self.rdata = rmodel.createData()

        self.refEff = pinocchio.SE3( rotate('y',np.pi/4),                 # Target orientation
                                     np.matrix([ -.3, 0.5, 0.2 ]).T)     # Target position
        self.idEff = rmodel.getFrameId('gripper_left_fingertip_2_link')
        self.refQ = rmodel.neutralConfiguration

        self.initDisplay(gview)
        
    def cost(self,x):
        q = a2m(x)
        self.residuals = m2a(q-self.refQ)
        return .5*sum(self.residuals**2)

    def dCost_dx(self,x):
        q = a2m(x)
        dq = m2a(q-self.refQ)
        return dq
    
    def constraint(self,x):
        q = a2m(x)
        pinocchio.forwardKinematics(self.rmodel,self.rdata,q)
        pinocchio.updateFramePlacements(self.rmodel,self.rdata)
        refMeff = self.refEff.inverse()*self.rdata.oMf[self.idEff]
        self.eq = m2a(pinocchio.log(refMeff).vector)
        return self.eq.tolist()

    def dConstraint_dx(self,x):
        q = a2m(x)
        pinocchio.forwardKinematics(self.rmodel,self.rdata,q)
        pinocchio.computeJointJacobians(self.rmodel,self.rdata,q)
        pinocchio.updateFramePlacements(self.rmodel,self.rdata)
        refMeff = self.refEff.inverse()*self.rdata.oMf[self.idEff]
        log_M = pinocchio.Jlog6(refMeff)
        M_q = pinocchio.getFrameJacobian(self.rmodel,self.rdata,self.idEff,LOCAL)
        self.Jeq = log_M*M_q
        return self.Jeq
        
    @property
    def bounds(self):
        # return [ (10*l,u) for l,u in zip(self.rmodel.lowerPositionLimit.flat,
        #                               self.rmodel.upperPositionLimit.flat) ]
        return [ (-10.,10) for i in range(self.rmodel.nq) ]
        
    def initDisplay(self,gview=None):
        self.gview = gview
        if gview is None: return
        self.gobj = "world/target6d"
        self.gview.addBox(self.gobj,.1,0.05,0.025,[1,0,0,1])
        self.gview.applyConfiguration(self.gobj,se3ToXYZQUAT(self.refEff))
        self.gview.refresh()

    def callback(self,x):
        import time
        q = a2m(x)
        robot.display(q)
        time.sleep(1e-2)


robot.q0 = robot.model.neutralConfiguration
pbm = OptimProblem(robot.model,robot.viewer.gui)

# --- NUMDIFF CHECK ------------------------------------
def numdiff(f,x,h=1e-6):
    f0 = f(x)
    nx,nf = len(x),len(f0)
    dx = np.zeros(nx)
    df_dx = np.zeros([nf,nx])
    for i in range(nx):
        dx[i] = h
        df_dx[:,i] = (f(x+dx)-f0)/h
        dx[i] = 0
    return df_dx

x = np.random.rand(robot.model.nq)*2-1

def costResiduals(x):
    pbm.cost(x)
    return pbm.residuals

assert( norm( pbm.dCost_dx(x) - np.dot( numdiff(costResiduals,x).T,costResiduals(x) ) ) <1e-6 )
assert( norm( pbm.dConstraint_dx(x) - numdiff(lambda x:np.array(pbm.constraint(x)),x) ) <1e-6 )

# --- NUMDIFF CHECK ------------------------------------

#x0  = np.random.rand(robot.model.nq)
x0 = np.array([ .7,.9,.8,.5,.9,.7,.1])

result = fmin_slsqp(x0       = x0,
                    func     = pbm.cost,
                    fprime   = pbm.dCost_dx,
                    f_eqcons = pbm.constraint,
                    fprime_eqcons = pbm.dConstraint_dx,
                    bounds   = pbm.bounds,
                    callback = pbm.callback)
qopt = a2m(result)


# Derivatives in T_q Q
If you want to go further, you can start investigate the derivatives in a configuration manifold (nonEuclidean) with this example.

In [23]:
#%load bip6d_diff.py